# Syndromic surveillance
1. Spatio-temporal analysis of data and clusters
2. Outbreak detection in form of outlier detection

In [1]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sb

# Import and prepare data

In [3]:
rw_data = pd.read_pickle("data/rw-cleaned-prepared-dynamic-data-phase1.pickle") #data after mandatory cleaning
labeled_preprocessed_features = pd.read_pickle("data/rwanda/final_labeled_preprocessed_features") # labeled preprocessed features
features = pd.read_pickle("data/rwanda/features.pickle") # cleaned and prepared features as selected by domain experts with missing values
diagnoses = pd.read_pickle("data/rwanda/diagnoses.pickle")#
labeled_features = features.copy()
labeled_features["cluster"] = labeled_preprocessed_features["cluster"]

In [4]:
# create spatio temporal clusters with human readable features
spatio_temporal_columns = [
    "medical_case_consultation_date_day", # time
    "health_facility_name", "health_facility_longitude", "health_facility_latitude",  #spatial variant 1 = hf location
    "District", "Sector", "Cell", "Village", "longitude_village", "latitude_village" #spatial variant 2 = patient's origin
]
spatio_temperol_clusters = pd.merge(features, rw_data[spatio_temporal_columns], left_index=True, right_index=True)
spatio_temperol_clusters["cluster"] = labeled_features["cluster"]

# preprocess selected cluster result for spatial temporal analysis:
# add time units day, week, month, year
from datetime import datetime

#input date is string in format "2022-01-01"
#output is week in format "2022-01"
def getWeek(date): 
    dateObj = datetime.strptime(date, "%Y-%m-%d")
    week = dateObj.strftime("%U")
    if len(week) == 1:
        return str(dateObj.year)+"-0"+str(week)
    else:
        return str(dateObj.year)+"-"+str(week)

# Extract the year, month and week from medical case consultation at day
# note: week starts from week 0
spatio_temperol_clusters["temp"] = [(date[0], date[0]+"-"+date[1], getWeek(date[0]+"-"+date[1]+"-"+date[2])) for date in spatio_temperol_clusters["medical_case_consultation_date_day"].str.split("-")]
spatio_temperol_clusters[["medical_case_consultation_date_year", "medical_case_consultation_date_month", "medical_case_consultation_date_week"]] = spatio_temperol_clusters["temp"].apply(pd.Series)
spatio_temperol_clusters = spatio_temperol_clusters.drop("temp", axis=1)

In [7]:
complaints_older_kids = [
    "CC21 - General - 8341",
    "CC32 - Fever - 8081",
    "CC13 - Respiratory (Cough or difficult breathing) - 7808",
    "CC17 - Gastrointestinal (diarrhea, vomiting, feeding) - 7875",
    "CC10 - Skin / hair - 8346",
    "CC11 - Eye - 7803",
    "CC12 - Ear/Throat/Mouth - 8342",
    "CC18 - Accident /Muskulo-skeletal (incl. burns, wounds, poison) - 8343",
    "CC15 - Genitourinary (UTI, STI) - 8371",
    "CC16 - Neuro (Headache, stiff neck, neck pain) - 8372",
    "CC35 - Other - 8120"]

complaints_infants = [
    "CC22 - General Assessment - 8352",
    "CC33 - Fever or convulsions or lethargy - 7391",
    "CC24 - Respiratory - 8370",
    "CC25 - Diarrhea, abdominal, gastro-intestinal - 8076",
    "CC27 - Eye complaint - 8066",
    "CC28 - Ear or mouth complaint - 8072",
    "CC23 - Yellow appearing skin or eyes (jaundice) - 7351",
    "CC30 - Malformation of birth anomaly - 7389",
    "CC29 - Feeding problem or weight concern - 8347",
    "CC31 - Injuries (birth and non-birth related) - 7977",
    "CC37 - Follow-up visit for a previous problem - 7521",
    "CC36 - Other - 8122"
]
complaint_data_infants = rw_data[complaints_infants]
complaint_data_kids = rw_data[complaints_older_kids]
complaints = rw_data[complaints_infants+complaints_older_kids]
labeled_complaints = complaints.copy()
labeled_complaints["cluster"] = labeled_features["cluster"]

In [35]:
# store for dash
spatio_temperol_clusters.to_pickle("data/dash/rw-spatio-temporal-cluster-data.pickle")
labeled_complaints.to_pickle("data/dash/labeled_complaints.pickle")

## Analysis of complaint categories

### General
Desired graph but due to time not implemented
![Desired graph but due to time not implemented](data/rwanda/example_complaint_category_overview_for_infants_or_kids.png)

### Complaints in clusters

In [10]:
complaint = complaints.columns[0]

In [13]:
#show distribution of upper selected feature per cluster depending on data type of feature
def show_complaint_distribution_per_cluster(complaint):
    cluster_size=labeled_complaints.groupby("cluster").size().reset_index(name="cluster_size")
    complaint_per_cluster=labeled_complaints.groupby(["cluster", complaint], dropna=False).size().reset_index(name="complaint_size")
    complaint_per_cluster=complaint_per_cluster.merge(cluster_size, on="cluster")
    complaint_per_cluster[complaint] = complaint_per_cluster[complaint].astype(str)
    complaint_per_cluster["cluster"] = complaint_per_cluster["cluster"]
    complaint_per_cluster["share_category"] = complaint_per_cluster["complaint_size"] / complaint_per_cluster["cluster_size"]
    fig = px.bar(complaint_per_cluster, x="cluster", y="share_category", color=complaint, title=f"Distribution of '{complaint}' per cluster", labels={"cluster":"Cluster", "share_category":"Cluster share", complaint:"Categories"})
    return fig

In [14]:
show_complaint_distribution_per_cluster(complaint).show()

## Demographic analysis of clusters

In [17]:
demographics=["patient_age", "patient_gender"]

In [18]:
#show distribution of upper selected feature per cluster depending on data type of feature
def show_feature_distribution_per_cluster(feature):
    if "float" in str(labeled_features[feature].dtype) and len(labeled_features[feature].dropna().unique())>2: #if continues feature plot as boxplots
        fig = px.box(labeled_features, x="cluster", y=feature, title=f"Distribution of '{feature}' per cluster", labels={"cluster":"Cluster"})
    else:
        cluster_size=labeled_features.groupby("cluster").size().reset_index(name="cluster_size")
        feature_per_cluster=labeled_features.groupby(["cluster", feature], dropna=False).size().reset_index(name="feature_size")
        feature_per_cluster=feature_per_cluster.merge(cluster_size, on="cluster")
        feature_per_cluster[feature] = feature_per_cluster[feature].astype(str)
        feature_per_cluster["cluster"] = feature_per_cluster["cluster"]
        feature_per_cluster["share_category"] = feature_per_cluster["feature_size"] / feature_per_cluster["cluster_size"]
        fig = px.bar(feature_per_cluster, x="cluster", y="share_category", color=feature, title=f"Distribution of '{feature}' per cluster", labels={"cluster":"Cluster", "share_category":"Cluster share", feature:"Categories"})
    return fig

In [19]:
for demographic in demographics:
    fig = show_feature_distribution_per_cluster(demographic)
    fig.write_image(f"plots/syndromic_surveillance/demographics/{demographic}.png")
    fig.show()

# Spatio-temporal cluster analysis

In [20]:
# see in dashboard

## Select space and time unit for spatio-temporal analysis
- space = patient's origin or hf location
- time = consultation date (day, week, month, year)

In [21]:
# select time units (week is default)
time = "medical_case_consultation_date_week" # or "medical_case_consultation_date_day", "medical_case_consultation_date_month", "medical_case_consultation_date_year"

# select space units (patient's origin is default)
space = ["District", "Sector", "Cell", "Village", "longitude_village", "latitude_village"] # or ["health_facility_longitude", "health_facility_latitude"]
space_coordinates = ["longitude_village", "latitude_village"] #or ["health_facility_name", "health_facility_longitude", "health_facility_latitude"]

# Outlier detection: select params

In [22]:
# summarize cluster data over space and time
consultations_per_cluster_over_space_time = spatio_temperol_clusters.groupby(space+[time, "cluster"]).size().reset_index(name="cluster_size")

# compute percentage of consultations per cluster over space and time
consultations_per_cluster_over_space_time["percentage_cluster_size"] = consultations_per_cluster_over_space_time["cluster_size"] / consultations_per_cluster_over_space_time.groupby(space+[time])["cluster_size"].transform('sum')

In [23]:
variable_to_be_checked_for_outliers = "percentage_cluster_size" #or "cluster_size"

## Outlier detection based on z-score

In [24]:
# compute upper bound for outliers based on z-score
z_score_threshold_of_cluster_at_space = consultations_per_cluster_over_space_time.groupby(space+["cluster"])[variable_to_be_checked_for_outliers].agg(["mean", "std"]).reset_index()

# check for outliers
z_score_outlier_detection = pd.merge(consultations_per_cluster_over_space_time, z_score_threshold_of_cluster_at_space, on=space+["cluster"])
z_score_outlier_detection["z_score"] = (z_score_outlier_detection[variable_to_be_checked_for_outliers] - z_score_outlier_detection["mean"]) / z_score_outlier_detection["std"]
z_score_outlier_detection["outlier"] = z_score_outlier_detection["z_score"] > 3 #is common chosen upper bound threshold

## Outlier detection based on IQR (=Inter Quartile Range)

In [25]:
# compute upper bound for outliers based on iqr
iqr_threshold_of_cluster_at_space = consultations_per_cluster_over_space_time.groupby(space+["cluster"]).agg(
    q1 = (variable_to_be_checked_for_outliers, lambda x: np.percentile(x, 25, method='midpoint')),
    q3 = (variable_to_be_checked_for_outliers, lambda x: np.percentile(x, 75, method='midpoint')),
).reset_index()
iqr_threshold_of_cluster_at_space["iqr"] = iqr_threshold_of_cluster_at_space["q3"] - iqr_threshold_of_cluster_at_space["q1"]
iqr_threshold_of_cluster_at_space["upper_bound"] = iqr_threshold_of_cluster_at_space["q3"] + 1.5 * iqr_threshold_of_cluster_at_space["iqr"]

# check for outliers
iqr_outlier_detection = pd.merge(consultations_per_cluster_over_space_time, iqr_threshold_of_cluster_at_space, on=space+["cluster"])
iqr_outlier_detection["outlier"] = iqr_outlier_detection["upper_bound"] < iqr_outlier_detection[variable_to_be_checked_for_outliers]

## Outlier detection based on Histogram-based Outlier Score (HBOS) of distribution of "number of consultations per space and time point"
Since space and time change so does the distribution.

follow this overview: https://www.dfki.de/fileadmin/user_upload/import/6431_HBOS-poster.pdf
alternatively check out: CBLOF [5] or LDCOF [1] which can be used after K-Means

In [26]:
distribution_percentage_cluster_size_at_space_point_over_time = consultations_per_cluster_over_space_time.groupby(space+["cluster", "percentage_cluster_size"]).size().reset_index(name="frequence")

""""fig = px.line(distribution_percentage_cluster_size_at_space_point_over_time, x="percentage_cluster_size", y="frequence", color="Village")
fig.show()"""

'"fig = px.line(distribution_percentage_cluster_size_at_space_point_over_time, x="percentage_cluster_size", y="frequence", color="Village")\nfig.show()'

##  Summarize outlier detection results

In [27]:
outlier_detection_datasets = [z_score_outlier_detection, iqr_outlier_detection]
outlier_detection_methods = ["z-score", "IQR"]

## Visualize outlier detection results over space and time

### Difference in outlier detection results

In [28]:
#iqr_outlier_detection.groupby(space+[time, "cluster"])["outlier"].sum().reset_index(name="number_outliers")
difference_outlier_detection_results = pd.merge(z_score_outlier_detection, iqr_outlier_detection, on=space+[time, "cluster"], suffixes=('_z_score', '_iqr'))

# space and time points where outlier detection methods disagree in outlier decision
difference_outlier_detection_results[~(difference_outlier_detection_results["outlier_z_score"] == difference_outlier_detection_results["outlier_iqr"])]

,District,Sector,Cell,Village,longitude_village,latitude_village,medical_case_consultation_date_week,cluster,cluster_size_z_score,percentage_cluster_size_z_score,...,std,z_score,outlier_z_score,cluster_size_iqr,percentage_cluster_size_iqr,q1,q3,iqr,upper_bound,outlier_iqr
36,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2022-52,2,1,1.0,...,0.203768,2.900812,False,1,1.0,0.291667,0.464286,0.172619,0.723214,True
100,nyamasheke,bushekeri,buvungira,buvungira,29.092868,-2.417103,2022-46,0,1,1.0,...,0.264663,1.899693,False,1,1.0,0.366667,0.500000,0.133333,0.700000,True
148,nyamasheke,bushekeri,buvungira,gisakura,29.090935,-2.434478,2022-37,2,1,1.0,...,0.273464,1.958992,False,1,1.0,0.250000,0.500000,0.250000,0.875000,True
156,nyamasheke,bushekeri,buvungira,gisakura,29.090935,-2.434478,2022-52,2,1,1.0,...,0.273464,1.958992,False,1,1.0,0.250000,0.500000,0.250000,0.875000,True
160,nyamasheke,bushekeri,buvungira,gisakura,29.090935,-2.434478,2022-45,0,1,0.5,...,0.252538,0.707107,False,1,0.5,0.321429,0.321429,0.000000,0.321429,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29766,rusizi,rwimbogo,muhehwe,murama,28.958341,-2.628608,2022-26,0,1,1.0,...,0.347691,1.764019,False,1,1.0,0.200000,0.333333,0.133333,0.533333,True
29910,rusizi,rwimbogo,muhehwe,renga,28.963583,-2.632096,2022-26,2,1,1.0,...,0.245624,1.992619,False,1,1.0,0.309524,0.583333,0.273810,0.994048,True
29914,rusizi,rwimbogo,muhehwe,renga,28.963583,-2.632096,2022-36,2,1,1.0,...,0.245624,1.992619,False,1,1.0,0.309524,0.583333,0.273810,0.994048,True
30059,rusizi,rwimbogo,mushaka,kabajoba,28.957216,-2.636876,2022-47,0,1,1.0,...,0.242097,2.202971,False,1,1.0,0.333333,0.500000,0.166667,0.750000,True


In [29]:
# comparison of number of outliers per cluster
difference_outlier_detection_results.groupby(["cluster"])["outlier_z_score", "outlier_iqr"].sum().reset_index()

C:\Users\sidla\AppData\Local\Temp\ipykernel_18328\4164376764.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,cluster,outlier_z_score,outlier_iqr
0,0,2,217
1,1,0,24
2,2,6,202


In [30]:
# comparison of number of outliers per cluster at space point
difference_outlier_detection_results.groupby([time, "cluster"])["outlier_z_score", "outlier_iqr"].sum().reset_index()

C:\Users\sidla\AppData\Local\Temp\ipykernel_18328\2113531865.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,medical_case_consultation_date_week,cluster,outlier_z_score,outlier_iqr
0,2021-48,0,0,1
1,2021-48,1,0,0
2,2021-48,2,0,0
3,2021-49,0,0,1
4,2021-49,1,0,0
...,...,...,...,...
184,2023-05,1,0,2
185,2023-05,2,0,6
186,2023-06,0,0,6
187,2023-06,1,0,1


In [31]:
# comparison of number of outliers per cluster at space point
difference_outlier_detection_results.groupby(space+["cluster"])["outlier_z_score", "outlier_iqr"].sum().reset_index()

C:\Users\sidla\AppData\Local\Temp\ipykernel_18328\3481002836.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,District,Sector,Cell,Village,longitude_village,latitude_village,cluster,outlier_z_score,outlier_iqr
0,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,0,0,0
1,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,1,0,0
2,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2,0,1
3,nyamasheke,bushekeri,buvungira,bushekeri,29.091001,-2.404073,1,0,0
4,nyamasheke,bushekeri,buvungira,bushekeri,29.091001,-2.404073,2,0,0
...,...,...,...,...,...,...,...,...,...
2796,rusizi,rwimbogo,ruganda,rubuye,29.003471,-2.647140,1,0,0
2797,rusizi,rwimbogo,ruganda,rubuye,29.003471,-2.647140,2,0,0
2798,rusizi,rwimbogo,ruganda,ruhinga,28.984867,-2.648469,0,0,0
2799,rusizi,rwimbogo,ruganda,ruhinga,28.984867,-2.648469,1,0,0


In [32]:
# comparison of number of outliers per cluster at space time point
difference_outlier_detection_results.groupby(space+[time, "cluster"])["outlier_z_score", "outlier_iqr"].sum().reset_index()

C:\Users\sidla\AppData\Local\Temp\ipykernel_18328\279109352.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,District,Sector,Cell,Village,longitude_village,latitude_village,medical_case_consultation_date_week,cluster,outlier_z_score,outlier_iqr
0,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2022-03,1,0,0
1,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2022-35,1,0,0
2,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2022-35,2,0,0
3,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2022-36,1,0,0
4,nyamasheke,bushekeri,buvungira,buhinga,29.080169,-2.418294,2022-37,1,0,0
...,...,...,...,...,...,...,...,...,...,...
30271,rusizi,rwimbogo,ruganda,ruhinga,28.984867,-2.648469,2023-04,0,0,0
30272,rusizi,rwimbogo,ruganda,ruhinga,28.984867,-2.648469,2023-04,2,0,0
30273,rusizi,rwimbogo,ruganda,ruhinga,28.984867,-2.648469,2023-05,1,0,0
30274,rusizi,rwimbogo,ruganda,ruhinga,28.984867,-2.648469,2023-05,2,0,0


### Outliers per cluster over space and time

In [33]:
# filter for outliers based on selected outlier detection method and then plot outliers over space and time
for i, outlier_detection_data in enumerate(outlier_detection_datasets):
    fig = px.scatter_mapbox(
        outlier_detection_data[outlier_detection_data["outlier"] == True], 
        lon=[c for c in outlier_detection_data.columns if "longitude" in c][0], 
        lat=[c for c in outlier_detection_data.columns if "latitude" in c][0], 
        color="cluster", 
        size="cluster_size",
        hover_data = outlier_detection_data.columns,
        center=dict(lon=30, lat=-2.2), zoom=7,
        mapbox_style="stamen-terrain",
        animation_frame = time,
        title="Detection outliers in 'consultation numbers per cluster' based on method: "+outlier_detection_methods[i],
        #range_color=(0, len(outlier_detection_data["cluster"].unique())-1),
        category_orders={"cluster": sorted(outlier_detection_data["cluster"].unique())}
    ).update_layout(
        showlegend=True,
        legend_title_text="Cluster"
    )
    """for i, c in enumerate(sorted(outlier_detection_data["cluster"].unique())):
        fig['data'][i]['showlegend']=True
        fig['data'][i]['name'] = "test"
    fig.show()
"""
    fig.show()

In [ ]:
# store outlier detection data for dashboard
outlier_detection_data.to_pickle("data/dash/outlier_detection_spatio_temporal.pickle")

# Analysis of outliers
- show symptom and diagnoses distribution of outliers

## Get composition of symptoms, clusters and diagnoses based on for selected space time point
Such an interactive plot will be of high utility for domain experts to further examine the outliers and thus validate them! 

In [34]:
# TODO bar chart for cluster_size/top 10 diagnoses/top 10 bnary features/boxplot contnues features/top 10 categorcal features with animation frame for time
# see in dashboard

## Takeaways
